In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp
import sklearn
#import tensorflow

In [2]:
mp_holistic = mp.solutions.holistic #makes detections
mp_drawing = mp.solutions.drawing_utils #draws detections

In [3]:
import numpy as np
import mediapipe as mp

def reconstruct_keypoints(flattened_array):
    # Number of landmarks and features
    num_pose_landmarks = 33
    #num_face_landmarks = 468
    num_hand_landmarks = 21
    num_pose_features = 4
    num_hand_features = 3
    num_face_features = 3
    
    # Determine the indices for slicing
    pose_end = num_pose_landmarks * num_pose_features
    #face_end = pose_end + num_face_landmarks * num_face_features
    lh_end = face_end + num_hand_landmarks * num_hand_features
    rh_end = lh_end + num_hand_landmarks * num_hand_features

    # Extract landmarks from flattened array
    pose_flat = flattened_array[:pose_end]
    #face_flat = flattened_array[pose_end:face_end]
    lh_flat = flattened_array[face_end:lh_end]
    rh_flat = flattened_array[lh_end:rh_end]

    # Reshape back to original format
    pose = np.reshape(pose_flat, (num_pose_landmarks, num_pose_features))
    face = np.reshape(face_flat, (num_face_landmarks, num_face_features))
    lh = np.reshape(lh_flat, (num_hand_landmarks, num_hand_features))
    rh = np.reshape(rh_flat, (num_hand_landmarks, num_hand_features))

    # Convert to list of tuples
    pose_landmarks = [{'x': pose[i, 0], 'y': pose[i, 1], 'z': pose[i, 2], 'visibility': pose[i, 3]} for i in range(num_pose_landmarks)]
    face_landmarks = [{'x': face[i, 0], 'y': face[i, 1], 'z': face[i, 2]} for i in range(num_face_landmarks)]
    lh_landmarks = [{'x': lh[i, 0], 'y': lh[i, 1], 'z': lh[i, 2]} for i in range(num_hand_landmarks)]
    rh_landmarks = [{'x': rh[i, 0], 'y': rh[i, 1], 'z': rh[i, 2]} for i in range(num_hand_landmarks)]

    return pose_landmarks, face_landmarks, lh_landmarks, rh_landmarks


In [4]:
def draw_landmarks_on_black(image_shape, face_landmarks, pose_landmarks, lh_landmarks, rh_landmarks):
    """Draw landmarks on a black background."""
    # Create a black image
    image = np.zeros(image_shape, dtype=np.uint8)
    
    # Helper function to draw landmarks
    def draw_landmark_list(landmarks, color):
        for landmark in landmarks:
            x = int(landmark['x'] * image_shape[1])
            y = int(landmark['y'] * image_shape[0])
            cv2.circle(image, (x, y), 3, color, -1)
    
    # Draw face landmarks
    if face_landmarks:
        draw_landmark_list(face_landmarks, (0, 255, 0))  # Green for face
    
    # Draw pose landmarks
    if pose_landmarks:
        draw_landmark_list(pose_landmarks, (255, 0, 0))  # Red for pose
    
    # Draw left hand landmarks
    if lh_landmarks:
        draw_landmark_list(lh_landmarks, (255, 255, 0))  # Yellow for left hand
    
    # Draw right hand landmarks
    if rh_landmarks:
        draw_landmark_list(rh_landmarks, (0, 255, 255))  # Cyan for right hand
    
    return image

# Example usage



In [5]:
image_shape = (480, 640, 3)
for j in range(60):
    for i in range(30):
        path = f"ISL_Data_New/Why/{j}"
        #Projects/ISL_Translator/ISL_Data/Teacher/0/0.npy
        frame = np.load(path+f"/{i}.npy")

        pose_landmarks, face_landmarks, lh_landmarks, rh_landmarks = reconstruct_keypoints(frame)

        plots = draw_landmarks_on_black(image_shape, face_landmarks, pose_landmarks, lh_landmarks, rh_landmarks)

        cv2.imshow('Landmarks', plots)
        cv2.waitKey(10)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cv2.destroyAllWindows()

ValueError: cannot reshape array of size 93 into shape (468,3)